<a href="https://colab.research.google.com/github/podyssea/RecommenderSystems/blob/main/RecSys_coursework_2021_2210049p.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assessed Coursework Template Notebook

This is the template notebook for the RecSys(H) 2021 coursework. It deals with data preparation and evaluation only.

Please note:
 - use H1 text headings for grouping together blocks of cells. You can then hide these while working on other blocks
 - leave the cell output visible when you submit the notebook



## Setup block

Setup the data files, Python etc.

In [ ]:
!rm -rf ratings* books* to_read* test*

!curl -o ratings.csv "http://www.dcs.gla.ac.uk/~craigm/recsysH/coursework/final-ratings.csv" 
!curl -o books.csv "http://www.dcs.gla.ac.uk/~craigm/recsysH/coursework/final-books.csv"
!curl -o to_read.csv "http://www.dcs.gla.ac.uk/~craigm/recsysH/coursework/final-to_read.csv"
!curl -o test.csv "http://www.dcs.gla.ac.uk/~craigm/recsysH/coursework/final-test.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7631k  100 7631k    0     0  2240k      0  0:00:03  0:00:03 --:--:-- 2240k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2366k  100 2366k    0     0   986k      0  0:00:02  0:00:02 --:--:--  986k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7581k  100 7581k    0     0  1719k      0  0:00:04  0:00:04 --:--:-- 1719k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1895k  100 1895k    0     0   799k      0  0:00:02  0:00:02 --:--:--  799k


In [ ]:
#Standard setup
import pandas as pd
import numpy as np
import torch
!pip install git+https://github.com/cmacdonald/spotlight.git@master#egg=spotlight
from spotlight.interactions import Interactions
SEED=42

# data preparation

In [ ]:
#load in the csv files
ratings_df = pd.read_csv("ratings.csv")
books_df = pd.read_csv("books.csv")
to_read_df = pd.read_csv("to_read.csv")
test = pd.read_csv("test.csv")

In [ ]:
#cut down the number of items and users
counts=ratings_df[ratings_df["book_id"] < 2000].groupby(["book_id"]).count().reset_index()
valid_books=counts[counts["user_id"] >= 10][["book_id"]]

books_df = books_df.merge(valid_books, on="book_id")
ratings_df = ratings_df[ratings_df["user_id"] < 2000].merge(valid_books, on="book_id")
to_read_df = to_read_df[to_read_df["user_id"] < 2000].merge(valid_books, on="book_id")
test = test[test["user_id"] < 2000].merge(valid_books, on="book_id")


#stringify the id columns
def str_col(df):
  if "user_id" in df.columns:
    df["user_id"] = "u" + df.user_id.astype(str)
  if "book_id" in df.columns:
    df["book_id"] = "b" + df.book_id.astype(str)

str_col(books_df)
str_col(ratings_df)
str_col(to_read_df)
str_col(test)



# Implicit

In [ ]:
to_read_df

,Unnamed: 0,user_id,book_id
0,386014,u1333,b1797
1,36194,u765,b1797
2,25568,u1042,b1797
3,596738,u1360,b1797
4,52814,u1782,b1797
...,...,...,...
135610,444981,u556,b1157
135611,428922,u343,b1157
135612,457472,u1193,b1743
135613,487831,u1121,b1743


#explicit

In [ ]:
ratings_df

,Unnamed: 0,user_id,book_id,rating
0,0,u1,b258,5
1,130,u11,b258,3
2,1998,u143,b258,4
3,4731,u325,b258,4
4,5510,u362,b258,2
...,...,...,...,...
124757,438735,u1920,b1499,4
124758,439348,u334,b1499,2
124759,439505,u927,b1499,4
124760,439728,u1298,b1499,4


#test

In [ ]:
test

,Unnamed: 0,user_id,book_id
0,26204,u978,b323
1,74616,u109,b323
2,334583,u1721,b323
3,297262,u429,b323
4,30956,u339,b323
...,...,...,...
33912,347299,u272,b1545
33913,1881,u126,b1333
33914,400826,u1841,b1513
33915,518727,u1599,b1513


#books

In [ ]:
books_df

,Unnamed: 0,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,0,b1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,1,b2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,2,b3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,3,b4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,4,b5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,1989,b1990,11331421,11331421,6504537,83,1590514637,9.781591e+12,"Jan-Philipp Sendker, Kevin Wiliarty",2002.0,Das Herzenhören,The Art of Hearing Heartbeats,eng,3.98,41647,50338,5650,704,2697,10330,19670,16937,https://images.gr-assets.com/books/1320437247m...,https://images.gr-assets.com/books/1320437247s...
1822,1990,b1991,8935689,8935689,14366,50,1857231384,9.781857e+12,Iain M. Banks,1987.0,Consider Phlebas,"Consider Phlebas (Culture, #1)",eng,3.85,48649,53499,2666,1083,3656,12772,20533,15455,https://images.gr-assets.com/books/1327951890m...,https://images.gr-assets.com/books/1327951890s...
1823,1992,b1993,31332,31332,2925979,63,345434803,9.780345e+12,Anne Rice,1998.0,The Vampire Armand,"The Vampire Armand (The Vampire Chronicles, #6)",en-US,3.75,54919,57566,756,1129,4866,17158,18670,15743,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
1824,1996,b1997,9565548,9565548,14452295,33,054762834X,9.780548e+12,Robin LaFevers,2012.0,Grave Mercy,"Grave Mercy (His Fair Assassin, #1)",eng,3.92,70476,74683,7661,2257,4487,15591,27223,25125,https://images.gr-assets.com/books/1320269319m...,https://images.gr-assets.com/books/1320269319s...


In [ ]:
books_df.sort_values('average_rating', ascending=False).head(5)

,Unnamed: 0,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
839,861,b862,17332218,17332218,16482835,34,765326361,9.780765e+12,Brandon Sanderson,2014.0,Words of Radiance,"Words of Radiance (The Stormlight Archive, #2)",eng,4.77,73572,108176,7261,154,404,2653,18272,86693,https://images.gr-assets.com/books/1391535251m...,https://images.gr-assets.com/books/1391535251s...
420,421,b422,862041,862041,2962492,76,545044251,9.780545e+12,J.K. Rowling,1998.0,Complete Harry Potter Boxed Set,"Harry Potter Boxset (Harry Potter, #1-7)",eng,4.74,190050,204125,6508,1105,1285,7020,30666,164049,https://images.gr-assets.com/books/1392579059m...,https://images.gr-assets.com/books/1392579059s...
1243,1307,b1308,17927395,17927395,25126749,39,NaN,NaN,Sarah J. Maas,2016.0,A Court of Mist and Fury,A Court of Mist and Fury (A Court of Thorns an...,eng,4.72,108384,126491,22700,931,1420,4725,18000,101415,https://images.gr-assets.com/books/1485259138m...,https://images.gr-assets.com/books/1485259138s...
974,1009,b1010,43070,43070,2244438,26,836218051,9.780836e+12,Bill Watterson,1988.0,The Essential Calvin and Hobbes: A Calvin and ...,The Essential Calvin and Hobbes: A Calvin and ...,en-US,4.65,93001,95543,598,717,895,5387,17566,70978,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
555,561,b562,7235533,7235533,8134945,48,765326353,9.780765e+12,Brandon Sanderson,2010.0,The Way of Kings,"The Way of Kings (The Stormlight Archive, #1)",eng,4.64,144822,164477,10947,1085,1827,7661,33962,119942,https://images.gr-assets.com/books/1388184640m...,https://images.gr-assets.com/books/1388184640s...


Here we construct the Interactions objects from `ratings.csv`, `to_read.csv` and `test.csv`. We manually specify the num_users and num_items parameters to all Interaction objects, in case the test set differs from your training sets.

In [ ]:
from collections import defaultdict
from itertools import count, combinations

from spotlight.cross_validation import random_train_test_split

iid_map = defaultdict(count().__next__)


rating_iids = np.array([iid_map[iid] for iid in ratings_df["book_id"].values], dtype = np.int32)
test_iids = np.array([iid_map[iid] for iid in test["book_id"].values], dtype = np.int32)
toread_iids = np.array([iid_map[iid] for iid in to_read_df["book_id"].values], dtype = np.int32)


uid_map = defaultdict(count().__next__)

test_uids = np.array([uid_map[uid] for uid in test["user_id"].values], dtype = np.int32)
rating_uids = np.array([uid_map[uid] for uid in ratings_df["user_id"].values], dtype = np.int32)
toread_uids = np.array([uid_map[iid] for iid in to_read_df["user_id"].values], dtype = np.int32)


uid_rev_map = {v: k for k, v in uid_map.items()}
iid_rev_map = {v: k for k, v in iid_map.items()}


rating_dataset = Interactions(user_ids=rating_uids,
                               item_ids=rating_iids,
                               ratings=ratings_df["rating"].values,
                               num_users=len(uid_rev_map),
                               num_items=len(iid_rev_map))

toread_dataset = Interactions(user_ids=toread_uids,
                               item_ids=toread_iids,
                               num_users=len(uid_rev_map),
                               num_items=len(iid_rev_map))

test_dataset = Interactions(user_ids=test_uids,
                               item_ids=test_iids,
                               num_users=len(uid_rev_map),
                               num_items=len(iid_rev_map))

print(rating_dataset)
print(toread_dataset)
print(test_dataset)

#here we define the validation set
toread_dataset_train, validation = random_train_test_split(toread_dataset, random_state=np.random.RandomState(SEED))

num_items = test_dataset.num_items
num_users = test_dataset.num_users

<Interactions dataset (1999 users x 1826 items x 124762 interactions)>
<Interactions dataset (1999 users x 1826 items x 135615 interactions)>
<Interactions dataset (1999 users x 1826 items x 33917 interactions)>


In [ ]:
print(toread_dataset_train)

<Interactions dataset (1999 users x 1826 items x 108492 interactions)>


In [ ]:
print(num_items)

1826


#Example code

To evaluate soem of your hand-implemented recommender systems (e.g. Q1, Q4), you will need to instantiate objects that match the specification of a Spotlight model, which `mrr_score()` expects.


Here is an example recommender object that returns 0 for each item, regardless of user.

In [ ]:
from spotlight.evaluation import mrr_score, rmse_score


class dummymodel:
  
  def __init__(self, numitems):
    self.predictions=np.zeros(numitems)
  
  #uid is the user we are requesting recommendations for;
  #returns an array of scores, one for each item
  def predict(self, uid):
    #this model returns all zeros, regardless of userid
    return( self.predictions )

#lets evaluate how the effeciveness of dummymodel

dummymodel(num_items)

# print(mrr_score(dummymodel(num_items), test_dataset, train=rating_dataset, k=100).mean())
#as expected, a recommendation model that gives 0 scores for all items obtains a MRR score of 0



In [ ]:

#note that the latest copy of Craig's Spotlight displays a progress bar if you set verbose=True
print(mrr_score(dummymodel(num_items), test_dataset, train=rating_dataset, k=100, verbose=True).mean())


1999it [00:00, 2968.30it/s]

0.0


#Question 1

Non personalised baselines for ranking books based on statistics

In [ ]:
print(ratings_df.groupby(["user_id"]).mean()["rating"].size)

1999


In [ ]:
print(ratings_df['rating'])

0         5
1         3
2         4
3         4
4         2
         ..
124757    4
124758    2
124759    4
124760    4
124761    3
Name: rating, Length: 124762, dtype: int64


In [ ]:
# class avg_rating:
#   def __init__(self, num_items):
#     self.predictions = np.ones(num_items) * ratings_df["rating"]
#     print(self.predictions)
 
#   def predict(self, uid):
#     # returns the avg rating regardless of user id
#     return (self.predictions)

# print(mrr_score(avg_rating(num_items), test_dataset, train=rating_dataset, k=100).mean())

In [ ]:
class total_ratings:
  def __init__(self, num_items):
    self.predictions = np.ones(num_items) * books_df["ratings_count"].sum()
    print(self.predictions)
 
  def predict(self, uid):
    # returns the avg rating regardless of user id
    return (self.predictions)

print(mrr_score(total_ratings(num_items), test_dataset, train=rating_dataset, k=100).mean())

[3.59758938e+08 3.59758938e+08 3.59758938e+08 ... 3.59758938e+08
 3.59758938e+08 3.59758938e+08]
0.0


In [ ]:
class total_5_ratings:
  def __init__(self, num_items):
    self.predictions = np.ones(num_items) * books_df["ratings_5"].sum()
    print(self.predictions)
 
  def predict(self, uid):
    # returns the avg rating regardless of user id
    return (self.predictions)

print(mrr_score(total_5_ratings(num_items), test_dataset, train=rating_dataset, k=100).mean())

[1.60841787e+08 1.60841787e+08 1.60841787e+08 ... 1.60841787e+08
 1.60841787e+08 1.60841787e+08]
0.0


In [ ]:
class total_5_ratings_frac:
  def __init__(self, num_items):
    self.predictions = np.ones(num_items) * (books_df['ratings_5'].sum()/books_df['ratings_count'].sum())
    print(self.predictions)
 
  def predict(self, uid):
    # returns the avg rating regardless of user id
    return (self.predictions)

print(mrr_score(total_5_ratings_frac(num_items), test_dataset, train=rating_dataset, k=100).mean())

[0.44708212 0.44708212 0.44708212 ... 0.44708212 0.44708212 0.44708212]
0.0


# Question 2

In [ ]:
#import necessary modules
from spotlight.interactions import Interactions
from spotlight.cross_validation import random_train_test_split
from spotlight.factorization.explicit import ExplicitFactorizationModel
from spotlight.factorization.implicit import ImplicitFactorizationModel
from collections import defaultdict
from itertools import count
import itertools
import time
from scipy.stats import rankdata

In [ ]:
#divide the dataset
explicit_train, explicit_test = random_train_test_split(rating_dataset, random_state=np.random.RandomState(SEED))

print(explicit_train)
print(explicit_test)

<Interactions dataset (1999 users x 1826 items x 99809 interactions)>
<Interactions dataset (1999 users x 1826 items x 24953 interactions)>


In [ ]:
#define the latent factors
latent_factors = [8,16,32,64]

In [ ]:
print("uid %d has user_id %s gave iid %d a rating of %d" % (explicit_train.user_ids[0], uid_rev_map.get(explicit_train.user_ids[0]), explicit_train.item_ids[0],explicit_train.ratings[0]))

uid 1056 has user_id u1560 gave iid 339 a rating of 5


In [ ]:
for latent in latent_factors:
    print("\nstarting training for latent factor %d......)\n" % latent)
    emodel = ExplicitFactorizationModel(n_iter=5,
                                    embedding_dim=latent, #this is Spotlight default
                                    use_cuda=False,
                                    random_state=np.random.RandomState(SEED) # ensure results are repeatable
    )

    emodel.fit(explicit_train)

    print("======== MRR ========= for latent factor %d" % latent)
    print("=====================================================)")
    print(mrr_score(emodel, explicit_test).mean())
    print("=====================================================)")
    print(mrr_score(emodel, explicit_test))
    print("=====================================================)")
    # print(rmse_score(emodel, validation).mean())


starting training for latent factor 8......)

======== MRR ========= for latent factor 8
=====================================================)
0.020076705019176255
=====================================================)
[0. 0. 0. ... 0. 0. 0.]
=====================================================)

starting training for latent factor 16......)

======== MRR ========= for latent factor 16
=====================================================)
0.021314625566751632
=====================================================)
[1. 0. 0. ... 0. 0. 0.]
=====================================================)

starting training for latent factor 32......)

======== MRR ========= for latent factor 32
=====================================================)
0.030269698341234108
=====================================================)
[0. 0. 0. ... 0. 0. 0.]
=====================================================)

starting training for latent factor 64......)

======== MRR ========= for latent factor 64
====

In [ ]:
for latent in latent_factors:
  print("\nstarting training for latent factor %d......)\n" % latent)
  imodel = ImplicitFactorizationModel(loss="bpr",n_iter=5, 
                                      embedding_dim=latent, #this is Spotlight default
                                      use_cuda=False,
                                      random_state=np.random.RandomState(SEED) # ensure results are repeatable
  )

  imodel.fit(explicit_train)

  print("======== MRR ========= for latent factor %d" % latent)
  print("=====================================================)")
  print(mrr_score(imodel, explicit_test).mean())
  print("=====================================================)")
  print(mrr_score(imodel, explicit_test))
  print("=====================================================)")
  # print(rmse_score(imodel, explicit_test).mean())


starting training for latent factor 8......)

======== MRR ========= for latent factor 8
=====================================================)
0.19221475817273717
=====================================================)
[0.         0.11111111 0.2        ... 0.         0.         0.1       ]
=====================================================)

starting training for latent factor 16......)

======== MRR ========= for latent factor 16
=====================================================)
0.19687145159881528
=====================================================)
[0.         0.25       0.33333333 ... 0.         1.         0.5       ]
=====================================================)

starting training for latent factor 32......)

======== MRR ========= for latent factor 32
=====================================================)
0.19242875405956947
=====================================================)
[0.  0.  0.2 ... 0.  0.5 0. ]
====================================================

# Question 3 (a)

In [ ]:
latent_factors = [8,16,32,64]

for latent in latent_factors:

  print("\nstarting training for latent factor %d......)\n" % latent)
  implicit_model = ImplicitFactorizationModel(loss="bpr",n_iter=5, 
                                              embedding_dim=latent, 
                                              use_cuda=False, 
                                              random_state=np.random.RandomState(SEED))
  
  implicit_model.fit(toread_dataset_train, verbose=False)
  implicit_model.fit(validation)

  print("======== MRR ========= for latent factor %d" % latent)
  print("=====================================================)")
  print(mrr_score(implicit_model, validation).mean())
  print("=====================================================)")
  print(mrr_score(implicit_model, validation))
  print("=====================================================)")
  # print(rmse_score(implicit_model, validation).mean())


starting training for latent factor 8......)

======== MRR ========= for latent factor 8
=====================================================)
0.18753583140776736
=====================================================)
[0.  0.  0.  ... 0.  0.  0.5]
=====================================================)

starting training for latent factor 16......)

======== MRR ========= for latent factor 16
=====================================================)
0.19200373996522072
=====================================================)
[0.         0.2        0.         ... 0.         0.         0.16666667]
=====================================================)

starting training for latent factor 32......)

======== MRR ========= for latent factor 32
=====================================================)
0.18776868593026674
=====================================================)
[0.  0.  0.  ... 0.  0.  0.2]
=====================================================)

starting training for latent factor 64

# Question 3 (b)

In [ ]:
# print("\nstarting training for latent factor %d......)\n")
# implicit_model = ImplicitFactorizationModel(loss="bpr",n_iter=5, 
#                                             embedding_dim=16, 
#                                             use_cuda=False, 
#                                             random_state=np.random.RandomState(SEED))

# implicit_model.fit(toread_dataset_train, verbose=False)

# print("======== MRR ========= for latent factor %d" % latent)
# print("=====================================================)")
# print(mrr_score(implicit_model, validation).mean())
# print("=====================================================)")
# print(mrr_score(implicit_model, validation))
# print("=====================================================)")

In [ ]:
# #find all the mrr scores of each user in the test data
# mrr_scores = mrr_score(implicit_model, validation)
# print('these are the mrr scores for each user in the validation set')
# print(mrr_scores)

# #find the position in the list of mrr scores where the RR is highest and take a slice of it
# maximumRR = max(mrr_scores)
# # print(maximumRR)
# print('these are the indexes in the mrr_scores where the user got maximum MRR')
# maximumRR_users_indexes = [i for i, j in enumerate(mrr_scores) if j == maximumRR][:2]
# print(maximumRR_users_indexes)

# print('these are the user_ids for each index with maximum MRR in the list')
# #find user ids with highest RR
# maximumRR_users_uids = toread_dataset_train.user_ids[maximumRR_users_indexes]
# print(maximumRR_users_uids)

In [ ]:
# # print(uid_map)
# print("\n====== users with highest RR scores =======\n")

# user_ids_for_max = []

# for id in range(0, len(maximumRR_users_uids)):
#   print("user with uid %d has user_id %s" % (maximumRR_users_uids[id], uid_rev_map.get(maximumRR_users_uids[id])))
#   user_ids_for_max.append(uid_rev_map.get(maximumRR_users_uids[id]))


# print(user_ids_for_max)

In [ ]:
# test_and_books = pd.merge(test, books_df, on=["book_id"])
# to_read_books = pd.merge(to_read_df, books_df, on=['book_id'])

In [ ]:
# def books_predicted(user, k):
#   predictions = np.argsort(imodel.predict(user))[:k]
#   predictions = rankdata(imodel.predict(user), method = "ordinal")[:k]
#   return [iid_rev_map.get(key) for key in predictions] 

In [ ]:
# def common_member(a, b): 
#     a_set = set(a) 
#     b_set = set(b) 
  
#     if (a_set & b_set): 
#         print(a_set & b_set) 
#     else: 
#         print("No common elements")

In [ ]:
# def actual_vs_predictions(user_id):
#   print("===== What did users placed on their shelf =====")
#   books = list(test_and_books.loc[test_and_books['user_id'] == user_id]['original_title'])
#   print("User with user_id %s placed on his shelve the following books:\n%s" % (user_id, books))

#   print("\n===== What users plan or wish to read on their shelf =====")
#   books_planned = list(to_read_books.loc[to_read_books['user_id'] == user_id]['original_title'])
#   print("User with user_id %s wishes to place on his shelve the following books:\n%s" % (user_id, books_planned))

#   print("\n===== System Predictions for user =====")
#   books_predicted(uid_map.get(user_id), len(books_df))
#   pred_list = list(set(to_read_books[to_read_books.book_id.isin(books_predicted(uid_map.get(user_id), len(books_df)))]['title']))
#   print("User with user_id %s was predicted to place on his shelve the following books:\n%s" % (user_id, pred_list))

#   print("\n===== Common Elements =====")
#   common_member(books_planned, pred_list)

In [ ]:
# for each_id in user_ids_for_max:
#   actual_vs_predictions(each_id)
#   print(implicit_model.predict(uid_map.get(each_id)))
#   print('=========================================================================================\n\n')

# Question 3 (b)

In [ ]:
# iids_idl = books_df.sort_values('average_rating', ascending=False).head(5)[['average_rating', 'book_id']]['book_id'].tolist()

In [ ]:
# from scipy.spatial.distance import pdist
# import sklearn.metrics.pairwise

# all_embeddings = []
# for each_iid in iids_idl:
#   embeddings = implicit_model.predict(iid_map.get(each_iid))
#   all_embeddings.append(embeddings)
#   print('=========================================================================================\n\n')

# print(pdist(all_embeddings).mean())

In [ ]:
books_df_replicate = books_df.copy()
books_df_replicate.rename(columns = {"Unnamed: 0" : "item_id"}, inplace = True)
books_df_replicate

,item_id,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,0,b1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,1,b2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,2,b3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,3,b4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,4,b5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,1989,b1990,11331421,11331421,6504537,83,1590514637,9.781591e+12,"Jan-Philipp Sendker, Kevin Wiliarty",2002.0,Das Herzenhören,The Art of Hearing Heartbeats,eng,3.98,41647,50338,5650,704,2697,10330,19670,16937,https://images.gr-assets.com/books/1320437247m...,https://images.gr-assets.com/books/1320437247s...
1822,1990,b1991,8935689,8935689,14366,50,1857231384,9.781857e+12,Iain M. Banks,1987.0,Consider Phlebas,"Consider Phlebas (Culture, #1)",eng,3.85,48649,53499,2666,1083,3656,12772,20533,15455,https://images.gr-assets.com/books/1327951890m...,https://images.gr-assets.com/books/1327951890s...
1823,1992,b1993,31332,31332,2925979,63,345434803,9.780345e+12,Anne Rice,1998.0,The Vampire Armand,"The Vampire Armand (The Vampire Chronicles, #6)",en-US,3.75,54919,57566,756,1129,4866,17158,18670,15743,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
1824,1996,b1997,9565548,9565548,14452295,33,054762834X,9.780548e+12,Robin LaFevers,2012.0,Grave Mercy,"Grave Mercy (His Fair Assassin, #1)",eng,3.92,70476,74683,7661,2257,4487,15591,27223,25125,https://images.gr-assets.com/books/1320269319m...,https://images.gr-assets.com/books/1320269319s...


# Question 3 (b) 2nd Solution

In [ ]:
def item_to_titles(item_ids):
  return books_df_replicate.loc[books_df_replicate["item_id"].isin(item_ids)]["title"]

def find_common_titles(a,b, predictions):
  previously_vs_predicted = item_to_titles(np.intersect1d(a, predictions))
  print("These titles were predicted to be previously shelved correctly")
  print(previously_vs_predicted)
  currently_vs_predicted = item_to_titles(np.intersect1d(b, predictions))
  print("These titles were predicted to be currently shelved correctly")
  print(currently_vs_predicted)
  


In [ ]:
results = {}
combos = list(combinations(range(3), 2))
shelved = ["previously","currently","predicted"]

for latent_factor in latent_factors:
  # print("\nstarting training for latent factor %d......)\n" % 16)
  implicit_model = ImplicitFactorizationModel(loss="bpr",n_iter=5, 
                                              embedding_dim=latent_factor, 
                                              use_cuda=False, 
                                              random_state=np.random.RandomState(SEED))

  implicit_model.fit(toread_dataset_train, verbose=False)
  implicit_model.fit(validation)

mrr_scores = mrr_score(implicit_model, test_dataset)
user_id = test_dataset.user_ids[mrr_scores.argmax()]

item_ids_indexes = np.where(toread_dataset.user_ids == user_id)
previously_shelved = toread_dataset.item_ids[item_ids_indexes_prev]

item_ids_indexes = np.where(test_dataset.user_ids == user_id)
currently_shelved = test_dataset.item_ids[item_ids_indexes_curr]

predictions = implicit_model.predict(user_id)
predicted_shelved = np.where(predictions > 0)[0]

shelved_items = [previously_shelved, currently_shelved, predicted_shelved]

find_common_titles(previously_shelved, currently_shelved, predicted_shelved)

These titles were predicted to be previously shelved correctly
0                 The Hunger Games (The Hunger Games, #1)
3                                   To Kill a Mockingbird
16                   Catching Fire (The Hunger Games, #2)
25                 The Da Vinci Code (Robert Langdon, #2)
44                                             Life of Pi
72                                The Host (The Host, #1)
76                                      Holes (Holes, #1)
79                                      The Little Prince
82                                   A Tale of Two Cities
84                                        The Giving Tree
104                             Allegiant (Divergent, #3)
106                                    A Walk to Remember
113                                  Tuesdays with Morrie
124                                                Hamlet
142                           All the Light We Cannot See
152            City of Ashes (The Mortal Instruments, #2)
172      

# Question 3c

In [ ]:
predictions_idl = implicit_model.predict(test_dataset.user_ids, test_dataset.item_ids)
top5 = predictions_idl.argsort()[:5]
top5_item_ids = test_dataset.item_ids[top5]
print(top5_item_ids)

[1327  162  460 1467  909]


# Question 5

# Question 6